Рейтинг сайтов для отзывов
iRecommend.ru — один из крупнейших сайтов отзывов в России и СНГ. Тут уже добавлено огромное количество
предприятий и товаров.
migReview.com — сервис позволяет оставить отзыв о компаниях, товарах, местах, еде, сайтах и многом другом в один клик.
otzovik.com — большая база уже написанных отзывов и разделов, но часто отзывы удаляют.
booking.com — крупнейший сервис с отзывами об отелях.
flamp.ru — проект от 2GIS, отзывы в основном про места и заведения.
market.yandex.ru — маркетплейс от яндекса с отзывами о товарах и интернет-магазинах подойдет лучше всего.
torg.mail.ru — еще один маркетплейс от крупного холдинга Майл.ру, по сути аналог Яндекс.Маркета.
yell.ru — сервис с отзывами о местах
torg.mail.ru — еще один маркетплейс от крупного холдинга Майл.ру, по сути аналог Яндекс.Маркета.
zoon.ru — на данном сайте в основном пишут отзывы о заведениях и местах

In [ ]:
import pyodbc
import re

In [ ]:
import re
import urllib.parse
import requests

In [ ]:
goods_list = [
    'AISINO A90','AQIS 5T 4G',
    'AQSI 5Ф','EVOTOR 5',
    'EVOTOR 72','EVOTOR 73',
    'САЛЮТ 12Ф'
]

In [ ]:
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:65.0) Gecko/20100101 Firefox/65.0"

In [ ]:
def parse_search_page(good_name, description_phrase):
    URL = "https://google.com/search?q={0}".format(
        urllib.parse.quote_plus(good_name + ' ' + description_phrase)
    )
    resp = requests.get(
        URL,
        headers={"user-agent": USER_AGENT}
    )
    response = str(resp.text) if resp.status_code==200 else ""
    with open('{}.html'.format(good_name),'w',encoding='utf-8') as file:
        file.write(resp.text)
    return response

In [ ]:
goods_links = dict()
for good in goods_list:
    res_text = parse_search_page(good, 'отзывы пользователей')
    soup = BeautifulSoup(res_text, 'lxml')
    res_links = [link.get('href') for link in soup.find_all('a') if str(link.get('href')).find('http')>-1]
    goods_links[good] = res_links

In [ ]:
goods_links

In [ ]:
!pip install selenium

In [1]:
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
driver = webdriver.Firefox('C:\\Driver')

In [3]:
def get_reviews_pages_links(start_link, product_name):
    # открываю стартовую страницу с помощью WebDriver
    driver.get(start_link)
    # нахожу поле ввода для поиска отзывов
    input_element = driver.find_element_by_id('header-search-input')
    # имитирую ввод интересующей информации в поле ввода
    input_element.send_keys(product_name)
    # нахожу кнопку для выполнения поиска
    button = driver.find_element_by_xpath('//*[@id="header"]/div[2]/div/form/div/input[2]')
    # имитирую нажатие на кнопку
    button.click()
    # когда произведен переход на страницу с результатами поиска отсекаю рекламу
    goods_reviews_elements = driver.find_elements_by_class_name('reviews-counter')
    # сохраняю ссылки на страницы с отзывами
    reviews_pages_links = [good.get_attribute("href") for good in goods_reviews_elements]
    return reviews_pages_links

In [4]:
reviews_links = []
def get_reviews_links(link):
    driver.get(link)
    # определяем элементы соответствующие превью отзыва
    elements = driver.find_elements_by_class_name('item-right')
    # проходим по элементам превью и собираем ссылки
    for element in elements:
        reviews_links.append(element.find_element_by_class_name('review-title').get_attribute("href"))
    # после сбора ссылок на странице, проверяем, есть ли еще страницы, если да - запускаем рекурсию
    next_page = driver.find_elements_by_css_selector('.pager-item.next.tooltip-top')
    if next_page:
        get_reviews_links(next_page[0].get_attribute("href"))

In [12]:
def parse_review(review_page_link):
    driver.get(review_page_link)
    element = driver.find_element_by_class_name("review-contents")
    # заголовок отзыва
    title = element.find_element_by_tag_name('h1').text.strip()
    # дата публикации отзыва
    review_date = element.find_element_by_class_name('postdate-line').text.split('\n')[0].strip()
    # положительные стороны товара
    plus = element.find_element_by_class_name('review-plus').text.strip()
    # отрицательные стороны товара
    minus = element.find_element_by_class_name('review-minus').text.strip()
    # текст отзыва
    description = element.find_element_by_css_selector('.review-body.description').text.strip()
    return title, review_date, plus, minus, description 

In [ ]:
connection_string = 'Driver={SQL Server Native Client 11.0};Server=DESKTOP-R9VI2A2\SQLEXPRESS;Database=replace_val;Trusted_Connection=yes;'

In [ ]:
def create_db_structure(self, connection_string, dbn):
    try:
        if not os.path.exists("C:\Databases"):
            os.mkdir("C:\Databases")
        f_connection = pyodbc.connect(re.sub('replace_val','master', connection_string), autocommit=True)
        createDbRequest = """CREATE DATABASE {0} ON PRIMARY (NAME=N'{1}',FILENAME=N'C:\\Databases\\{2}.mdf') LOG ON (NAME=N'{3}_log',FILENAME=N'C:\\Databases\\{4}_log.ldf')""".format(dbn, dbn, dbn, dbn, dbn)
        with f_connection:
            f_cursor = f_connection.cursor()
            f_cursor.execute(createDbRequest)
        s_connection = pyodbc.connect(re.sub('replace_val', dbn, connection_string), autocommit=True)
        createTableRequest = """CREATE TABLE postsInfo(groupID int,postID int,postDate datetime,postText nvarchar(max),postStatus bit)"""
        with s_connection:
            s_cursor = s_connection.cursor()
            s_cursor.execute(createTableRequest)        
    except Exception as error:
        print("Method failed with error:" + str(error))

In [ ]:
def insertDataInCommentsInfo(self,commentId,commentatorId,postID,postDate,commentText):
    try:
        sqlString = """
        INSERT INTO commentsInfo (commentId,commentatorId,postID,postDate,commentText,userStatus,textStatus) VALUES(?,?,?,?,?,0,0)"""
        connection = pyodbc.connect(connectionString)
        dbCursor = connection.cursor()
        dbCursor.execute(sqlString, commentId,commentId,commentatorId,postID,postDate,re.sub('\n',' ',commentText))
        connection.commit()
        connection.close()
    except Exception as error:
        print("Error in insertDataInCommentsInfo " +str(error))

In [6]:
link = 'https://otzovik.com/'
product_name = 'aqsi'

In [7]:
# найти ссылки страниц со списком отзывов
reviews_pages_links = get_reviews_pages_links(link, product_name)

In [8]:
# найти ссылки страниц с отзывами
for review_page_link in reviews_pages_links:
    get_reviews_links(review_page_link)

In [13]:
# загрузить отзывы
with open('reviews.csv','w') as reviews_file:
    for link in reviews_links:
        try:
            title, review_date, plus, minus, description  = parse_review(link)
            reviews_file.write('{0};{1};{2};{3};{4};\n'.format(title, review_date, plus, minus, description))
        except Exception as error:
            print(error)

Message: TimedPromise timed out after 300000 ms
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:183:5
TimeoutError@chrome://remote/content/shared/webdriver/Errors.jsm:472:5
bail@chrome://remote/content/marionette/sync.js:227:19



In [ ]:
driver.quit()